In [1]:
%load_ext dotenv
%dotenv ../.env

In [4]:
import os
from klaviyo_api import KlaviyoAPI

In [5]:
klaviyo = KlaviyoAPI(os.environ["KLAVIYO_API_KEY"])

In [6]:
klaviyo.Metrics.get_metrics() 

{'data': [{'type': 'metric',
   'id': 'R6pkzd',
   'attributes': {'name': 'Bounced Email',
    'created': '2024-04-29T21:05:18+00:00',
    'updated': '2024-04-29T21:05:18+00:00',
    'integration': {'object': 'integration',
     'id': '0rG4eQ',
     'key': 'klaviyo',
     'name': 'Klaviyo',
     'category': 'Internal'}},
   'links': {'self': 'https://a.klaviyo.com/api/metrics/R6pkzd/'}},
  {'type': 'metric',
   'id': 'RHqpbQ',
   'attributes': {'name': 'Opened Email',
    'created': '2024-04-29T21:05:18+00:00',
    'updated': '2024-04-29T21:05:18+00:00',
    'integration': {'object': 'integration',
     'id': '0rG4eQ',
     'key': 'klaviyo',
     'name': 'Klaviyo',
     'category': 'Internal'}},
   'links': {'self': 'https://a.klaviyo.com/api/metrics/RHqpbQ/'}},
  {'type': 'metric',
   'id': 'S6xELh',
   'attributes': {'name': 'Received Automated Response SMS',
    'created': '2024-04-29T21:05:22+00:00',
    'updated': '2024-04-29T21:05:22+00:00',
    'integration': {'object': 'integra

In [17]:
klaviyo.Campaigns.get_campaigns("equals(messages.channel,'email')")

{'data': [{'type': 'campaign',
   'id': '01HWNTWW47VMBR99KEDW5N9YBR',
   'attributes': {'name': 'Internal Klaviyo - Test Campaign Name',
    'status': 'Sent',
    'archived': False,
    'audiences': {'included': ['WgYJhH'], 'excluded': []},
    'send_options': {'use_smart_sending': False, 'ignore_unsubscribes': False},
    'tracking_options': {'is_add_utm': False,
     'utm_params': [],
     'is_tracking_clicks': True,
     'is_tracking_opens': True},
    'send_strategy': {'method': 'immediate',
     'options_static': None,
     'options_throttled': None,
     'options_sto': None},
    'created_at': '2024-04-29T21:06:43.464192+00:00',
    'scheduled_at': None,
    'updated_at': '2024-04-29T21:06:43.464231+00:00',
    'send_time': None},
   'relationships': {'campaign-messages': {'data': [{'type': 'campaign-message',
       'id': '01HWNTWW4CNE8MH82QZDTD4EE9'}],
     'links': {'self': 'https://a.klaviyo.com/api/campaigns/01HWNTWW47VMBR99KEDW5N9YBR/relationships/campaign-messages/',
     

### Create a template using the newletter archive in the database

In [18]:
from prisma import Prisma
db = Prisma()
await db.connect()

In [19]:
emails = await db.email.find_many(order=
    {
        "createdAt": "desc",
    }, take=10)

In [21]:
e = emails[3]

In [22]:
e.name

'April_07_2024_Newsletter'

In [27]:
body = {
    "data": {
        "type": "template",
        "attributes": {
          "name": e.name,
          "editor_type": "CODE",
          "html": e.html,
          "text": e.text
        }
    }
}

In [28]:
resp = klaviyo.Templates.create_template(body)

In [44]:
template = "RqjrFW" # resp["data"]["id"]

In [29]:
resp

{'data': {'type': 'template',
  'id': 'RqjrFW',
  'attributes': {'name': 'April_07_2024_Newsletter',
   'editor_type': 'CODE',
   'html': '<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml"><head>\n<title>\n</title>\n<!--[if !mso]><!-->\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<!--<![endif]-->\n<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n<meta content="width=device-width, initial-scale=1" name="viewport"/>\n<!--[if mso]>\n        <noscript>\n        <xml>\n        <o:OfficeDocumentSettings>\n          <o:AllowPNG/>\n          <o:PixelsPerInch>96</o:PixelsPerInch>\n        </o:OfficeDocumentSettings>\n        </xml>\n        </noscript>\n        <![endif]-->\n<!--[if lte mso 11]>\n        <style type="text/css" data-inliner="ignore">\n          .mj-outlook-group-fix { width:100% !important; }\n        </style>\n        <![endif]-->\n<!--[if !mso]><

OK great, now you can see the template here, under the "Saved templates" tab https://www.klaviyo.com/templates/list

### Create a Campaign using the template

In [34]:
lists = klaviyo.Lists.get_lists(filter="equals(name,'Sample Data List')")

In [35]:
lists

{'data': [{'type': 'list',
   'id': 'WgYJhH',
   'attributes': {'name': 'Sample Data List',
    'created': '2024-04-29T21:06:41+00:00',
    'updated': '2024-04-29T21:06:41+00:00',
    'opt_in_process': 'double_opt_in'},
   'relationships': {'profiles': {'links': {'self': 'https://a.klaviyo.com/api/lists/WgYJhH/relationships/profiles/',
      'related': 'https://a.klaviyo.com/api/lists/WgYJhH/profiles/'}},
    'tags': {'links': {'self': 'https://a.klaviyo.com/api/lists/WgYJhH/relationships/tags/',
      'related': 'https://a.klaviyo.com/api/lists/WgYJhH/tags/'}}},
   'links': {'self': 'https://a.klaviyo.com/api/lists/WgYJhH/'}}],
 'links': {'self': 'https://a.klaviyo.com/api/lists/?filter=equals%28name%2C%27Sample%20Data%20List%27%29',
  'next': None,
  'prev': 'https://a.klaviyo.com/api/lists/?filter=equals%28name%2C%27Sample+Data+List%27%29&page%5Bcursor%5D=%3CFilters%3A+equals%28name%2C%27Sample+Data+List%27%29%3E'}}

In [41]:
body = {
    "data": {
        "type": "campaign",
        "attributes": {
            "name": e.name,
            "audiences": {
                "included": [lists["data"][0]["id"]],
                "send_strategy": "immediate",
            },
            "campaign-messages": {
                "data": [
                    {
                        "type": "campaign-message",
                        "attributes": {
                            "channel": "email",
                            "label": "My message name",
                            "content": {
                                "subject": "Buy our product!",
                                "preview_text": "My preview text",
                                "from_email": "store@my-company.com",
                                "from_label": "My Company",
                                "reply_to_email": "reply-to@my-company.com",
                                "cc_email": "cc@my-company.com",
                                "bcc_email": "bcc@my-company.com",
                            },
                            "render_options": {
                                "shorten_links": True,
                                "add_org_prefix": True,
                                "add_info_link": True,
                                "add_opt_out_language": False,
                            },
                        },
                    }
                ]
            },
        },
    }
}

In [43]:
resp = klaviyo.Campaigns.create_campaign(body)

Associate the campaign with a template

In [47]:
resp

{'data': {'type': 'campaign',
  'id': '01HX7509JEYZ1EJFZ3MM405XDF',
  'attributes': {'name': 'April_07_2024_Newsletter',
   'status': 'Draft',
   'archived': False,
   'audiences': {'included': ['WgYJhH'], 'excluded': []},
   'send_options': {'use_smart_sending': True, 'ignore_unsubscribes': False},
   'tracking_options': {'is_add_utm': False,
    'utm_params': [],
    'is_tracking_clicks': True,
    'is_tracking_opens': True},
   'send_strategy': {'method': 'immediate',
    'options_static': None,
    'options_throttled': None,
    'options_sto': None},
   'created_at': '2024-05-06T14:30:26.639220+00:00',
   'scheduled_at': None,
   'updated_at': '2024-05-06T14:30:26.699747+00:00',
   'send_time': None},
  'relationships': {'campaign-messages': {'data': [{'type': 'campaign-message',
      'id': '01HX7509JJ4PYJWH2D59FT2JPX'}],
    'links': {'self': 'https://a.klaviyo.com/api/campaigns/01HX7509JEYZ1EJFZ3MM405XDF/relationships/campaign-messages/',
     'related': 'https://a.klaviyo.com/a

In [54]:
klaviyo.Campaigns.get_campaigns(filter="equals(messages.channel,'email')")

{'data': [{'type': 'campaign',
   'id': '01HX7509JEYZ1EJFZ3MM405XDF',
   'attributes': {'name': 'April_07_2024_Newsletter',
    'status': 'Draft',
    'archived': False,
    'audiences': {'included': ['WgYJhH'], 'excluded': []},
    'send_options': {'use_smart_sending': True, 'ignore_unsubscribes': False},
    'tracking_options': {'is_add_utm': False,
     'utm_params': [],
     'is_tracking_clicks': True,
     'is_tracking_opens': True},
    'send_strategy': {'method': 'immediate',
     'options_static': None,
     'options_throttled': None,
     'options_sto': None},
    'created_at': '2024-05-06T14:30:26.639220+00:00',
    'scheduled_at': None,
    'updated_at': '2024-05-06T14:30:26.699747+00:00',
    'send_time': None},
   'relationships': {'campaign-messages': {'data': [{'type': 'campaign-message',
       'id': '01HX7509JJ4PYJWH2D59FT2JPX'}],
     'links': {'self': 'https://a.klaviyo.com/api/campaigns/01HX7509JEYZ1EJFZ3MM405XDF/relationships/campaign-messages/',
      'related': '

In [55]:
campaign = "01HX7509JJ4PYJWH2D59FT2JPX" #resp["data"]["relationships"]["campaign-messages"]["data"][0][["id"]

In [56]:
body =  {
    "data": {
        "type": "campaign-message",
        "id": campaign,
        "relationships": {"template": {"data": {"type": "template", "id": template}}},
    }
}

In [58]:
klaviyo.Campaigns.create_campaign_message_assign_template(body)